# mimikit Snippets <a id="TOP" name="TOP"></a>

This is a collection of small self-contained code snippets that you can copy & paste in your notebook.


> **Note**: This notebook is only for reference and not to be evaluated as is.

For an example of how to build your own notebook, checkout the [base `FreqNet` notebook]()

For documentation on the freqnet package and its classes checkout [this link]()


## Table of Content

- O. [Setup](#O.)
    - a) [Installation](#Oa)
    - b) [connect to Gdrive (colab)](#Ob)
    - c) [setup neptune](#Oc)
    - d) [variables](#Od)


- I. [Database](#I.)
    - a) [make a Database](#Ia)
    - b) [get data from a database](#Ib)


- II. [Training](#II.)
    - a) [configure](#IIa)
    - b) [train](#IIb)
    - c) [plot the losses](#IIc)
    - d) [open tensorboard](#IId)
    - e) [load checkpoint](#IIe)
    - f) [resume training](#IIf)
    - g) [manually save checkpoint](#IIg)


- III. [Sampling](#III.)
    - a) [generate](#IIIa)
    - b) [manual prompt](#IIIb)
    - c) [automatic prompt](#IIIc)
    - d) [random prompt](#IIId)
    - e) [same prompt(s) for several models](#IIIe)
    - f) [log audio](#IIIf)
    - g) [plot the outputs](#IIIg)
    


- IV. [Neptune](#IV.)
    - a) [configure](#IVa)
    - b) [available methods](#IVb)
    - c) [access Experiment object after training](#IVc)


# Setup <a id="O."></a>

## a) Installation <a id="Oa"></a><a name="Oa"></a>

In [ ]:
# on your PC in a terminal :
pip install mimikit

# in any notebook (colab or otherwise) :
!pip install mimikit

[Back to the top](#TOP)

## b) connect to GDrive (colab) <a id="Ob"></a><a name="Ob"></a>
this is pasted from the colab snippets

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive

[Back to the top](#TOP)

## c) setup neptune <a id="Oc"></a><a name="Oc"></a>

this shows how to setup neptune within a noteboook. For how to setup neptune on your computer, check out the [Step-by-step neptune notebook](https://colab.research.google.com/github/k-tonal/mimikit-notebooks/blob/main/Step-by-step%20neptune.ipynb) 

In [ ]:
from mimikit import NeptuneConnector

nep_con = NeptuneConnector(user="MimiK_it",
                           setup={"db": "mimikit-demo/MMK-03",
                                  "model": "mimikit-demo"})

## d) variables <a id="Od"></a> <a name="Od"></a>

Throughout the snippets we use consistent variable names.

Here we pseudo-exemplify them for reference.

It is _strongly advised_ to have a similar cell at the begining of your notebook with your own values and "build" your notebook through copying, pasting & adapting the snippets listed below.

This will protect you against typos and mistakes and allow you to use the notebooks' cells as small modules within a well-defined environment.

In [ ]:
path_to_audio_files = "MyPc/home/my_audios"

path_to_db = "directory/db.h5"

# where files are stored :
path_to_model = "home/mimikit-models/"

path_to_ckpt = path_to_model + "/states/epoch=X.ckpt"

# 3 main objects are db, model and trainer :
db = Database(path_to_db)
model = ModelClass(....)
trainer = get_trainer(root_dir=path_to_model, ....)


[Back to the top](#TOP)

# I. BUILD AND USE `Database`<a id="I."></a><a name="I."></a>

## a) Make a DB <a id="Ia"></a><a name="Ia"></a>

#### Option 1 : with a python function

In [ ]:
from mimikit.data import freqnet_db

freqnet_db(path_to_db,
           roots=["directory/"],
          files=["file1", "file2"],
           # those are the defaults and can be omitted :
          n_fft=2048,
          hop_length=512,
          sample_rate=22050,
           # specifying a neptune_path ("account/project" and optionally + "/EXP-ID") 
           # automatically upload the db to this path (either updating an existing experiment 
           # or creating a new one)
          neptune_path=None)

#### Option 2 : with a command-line (once you pip installed mimikit)

in a terminal, after having installed mimikit
this takes exactly the same arguments as the function in Option 1 :


In [ ]:
freqnet-db "new_db.h5" --roots "directory/" --files "file1" "file2" --n-fft 1024 --hop-length 256

for usage and flags shorthands type :

In [ ]:
freqnet-db --help

#### Option 3 : manually build a FileWalker and an extract function! chekout mimikit.freqnet.freqnet_db for an example!

[Back to the top](#TOP)

## b) get data from a db <a id="Ib"></a><a name="Ib"></a>

`Database` keeps the files flatten in a large array under a feature attribute (in `freqnet_db`s the FFTs are in the `db.fft` attribute) which can be indexed like a `numpy` array.

In addition, the positions of each file in the flatten array is stored in `db.metadata`, a `pandas.Dataframe` with the `columns` `["name", "start", "stop", "duration"]`. Each row of `db.metadata` corresponds to one extracted file.

In [ ]:
from mimikit.data import Database

db = Database(path_to_db)

# assuming the db is a freqnet-db,
# retrieve from frame 5 to frame 10 :

frames = db.fft[5:10]

# retrieve a whole file :

file_array = db.fft.get(db.metadata.iloc[[0]])

[Back to the top](#TOP)

# II. Training <a id="II."></a><a name="II."></a>

## a) configure <a id="IIa"></a><a name="IIa"></a>

`get_trainer()`

In [ ]:
from mimikit import get_trainer

trainer = get_trainer(# where to store the files :
                      root_dir=path_to_model,
                      # how often to save checkpoint :
                      epochs=10,
                      # train for how many epochs :
                      max_epochs=50)

# if you configured neptune just add 2 arguments :

trainer = get_trainer(# where to store the files :
                      root_dir=path_to_model,
                      # how often to save checkpoint :
                      epochs=10,
                      # train for how many epochs :
                      max_epochs=50,
                      # model and NeptuneConnector :
                      model=model,
                      neptune_connector=nep_con)

[Back to the top](#TOP)

## b) train <a id="IIb"></a><a name="IIb"></a>

In [ ]:
trainer.fit(model)

[Back to the top](#TOP)

## c) plot the losses <a id="IIc"></a><a name="IIc"></a>

In [ ]:
# TODO

[Back to the top](#TOP)

## d) open tensorboard <a id="IId"></a><a name="IId"></a>

assuming `path_to_model="./"` and you are in a notebook

In [ ]:
!tensorboard --logdir ./logs

## e) load checkpoint <a id="IIe"></a><a name="IIe"></a>

In [ ]:
# IMPORTANT : all models in mimikit.freqnet need a data_object argument when reloading

from mimikit.freqnet import FreqNet
from mimikit.data import Database

db = Database(path_to_db)
model = FreqNet.load_from_checkpoint(path_to_ckpt, data_object=db.fft)

[Back to the top](#TOP)

## f) resume training from checkpoint <a id="IIf"></a><a name="IIf"></a>

In [ ]:
# import the class you want to load

from mimikit.freqnet import FreqNet
from mimikit import get_trainer


# FreqNetModel needs a valid data_object when loading :

model = FreqNet.load_from_checkpoint(path_to_ckpt, data_object=db.fft)

# this resume training from a model checkpoint (path_to_ckpt) 
# and the last_optim_state.pt in the states/ directory

trainer = get_trainer(model=model,
                      resume_from_checkpoint=path_to_ckpt,
                      # if ckpt was made at epoch=25, 
                      # the following will train for 25 more epochs:
                      max_epochs=50,
                      epochs=10,
                      root_dir=path_to_model,
                      )

# trainer.fit(fnet)

[Back to the top](#TOP)

## g) manually save checkpoint <a id="IIg" name="IIg"></a>

In [ ]:
import os

# add a checkpoint in the model's states/ directory (where they belong...)

trainer.save_checkpoint(os.path.join(trainer.default_root_dir, "states", "test.ckpt"))

[Back to the top](#TOP)

# III. Sampling <a id="III." name="III."></a>

## a) generate <a id="IIIa" name="IIIa"></a>

the cells below show you how to define `prompt`.

In [ ]:
output = model.generate(prompt, n_steps=2048,
                        hop_length=db.fft.attrs["hop_length"])

[Back to the top](#TOP)

## b) manual prompt <a id="IIIb" name="IIIb"></a>

`Database` stores all files in flat array. Informations about the files you extracted are stored in `db.metadata` which is a `panda.Dataframe`.
You can then pick prompts anywhere in the db or in specific files

In [ ]:
db = Database(path_to_db)

# in all cases we get the length of the prompt from the model 

prompt_length = model.receptive_field

# anywhere in the db :

# this need to be smaller than db.fft.shape[0] - prompt_length
start_index = 1234

# in a specific file :

file = db.metadata.iloc[[3]]
# start index relative to the beginning of the file
start_index = 1234 + file["start"].item()

# finally 

prompt = db.fft[start_index:start_index+prompt_length]

[Back to the top](#TOP)

## c) automatic prompts <a id="IIIc" name="IIIc"></a>
simple trick to sample throughout a db

In [ ]:
db = Database(path_to_db)
N = db.fft.shape[0]
prompt_length = model.receptive_field

# how many prompts you want
n_prompts = 40

for i in range(0, N, N // n_prompts):
    prompt = db.fft[i:i+prompt_length]

#     then most likely :
#     output = model.generate(prompt, 1000, hop_length=db.fft.attrs["hop_length"])
#     model.log_audio(output, "prompt=%i" % i, sample_rate=db.fft.attrs["sr"])

[Back to the top](#TOP)

## d) random prompt <a id="IIId" name="IIIVd"></a>

In [ ]:
# all freqnets have methods to get random train and val examples :

# train :

prompt = model.random_train_example()

# validation :

prompt = model.random_val_example()

[Back to the top](#TOP)

## e) same prompt(s) for several models <a id="IIIe" name="IIIe"></a>

In [ ]:
# just as an example :

from random import randint
import os

db = Database(path_to_db)
n_prompts = 10
prompts_indices = [randint(0, db.fft.shape[0]) for _ in range(n_prompts)]
# how many steps we want to generate :
n_steps = 1000

all_checkpoints = os.listdir(os.path.join(path_to_model, "states"))

for ckpt in all_checkpoints:
    # load the model
    path_to_ckpt = os.path.join(path_to_model, "states", ckpt)
    model = FreqNet.load_from_checkpoint(path_to_ckpt)

    # loop through the prompts 
    for i in prompts_indices:
        prompt = db.fft[i:i+model.receptive_field]
        output = model.generate(prompt, n_steps, hop_length=db.fft.attrs["hop_length"])
        model.log_audio(output, "prompt=%i" % i, sample_rate=db.fft.attrs["sr"])
        

[Back to the top](#TOP)

## f) log audio <a id="IIIf"></a><a name="IIIf"></a>

These method will always save an audio file in `path_to_model/audios`
& in any neptune or TestTube loggers / experiments if the model is bound to some or if you passed some through
the `experiments=[...]` keyword argument

In [ ]:
# some audio tensor as reutrned by FreqNet.generate(...)
audio_tensor = torch.randn(1, 10000)
audio_filename = "name_of_the_file_you_want_to_create"

# if model just have been trained with some loggers:
model.log_audio(audio_filename, audio_tensor)

# if the data was made with non-default sample-rate :
model.log_audio(audio_filename, audio_tensor, sample_rate=db.fft.attrs["sr"])

# if you want to log to specific Experiment objects (neptune or testtube)
model.log_audio(audio_filename, audio_tensor, experiments=[neptune_exp])

[Back to the top](#TOP)

## g) plot the outputs <a id="IIIg" name="IIIg"></a>

#### 1. plot in time domain

In [ ]:
import matplotlib.pyplot as plt
from mimikit.utils import audio
from librosa.display import waveplot

prompt = model.random_train_example()

output = model.generate(prompt, n_steps=2048,
                        hop_length=db.fft.attrs["hop_length"])


plt.figure(figsize=(18, 6))
waveplot(output.squeeze().numpy())

audio(output.squeeze().numpy())

#### 2. plot in freq domain
(Be careful not to mix up the outputs variable afterwards!)

In [ ]:
import matplotlib.pyplot as plt
from mimikit.utils import audio, show, signal


prompt = model.random_train_example()

freq_output = model.generate(prompt, n_steps=2048,
                        hop_length=db.fft.attrs["hop_length"],
                        time_domain=False).squeeze().cpu().numpy().T

output = signal(freq_output)

plt.figure(figsize=(24, 6))
show(freq_output)

audio(output)

[Back to the top](#TOP)

# IV. Neptune <a id="IV."></a><a name="IV."></a>

## a) configure <a id="IVa"></a><a name="IVa"></a>

In [ ]:
from mimikit import NeptuneConnector

# for downloading a db and creating a model experiment :

nep_con = NeptuneConnector(user="mimi",
                           setup=dict(db="data/DAT-01",
                                      model="freqnets"))

# `setup` is mutable :

nep_con.setup["model"] += "/FRQ-10"

# now you would have this exact setup :

nep_con = NeptuneConnector(user="mimi",
                           setup=dict(db="data/DAT-01",
                                      model="freqnets/FRQ-10"))

[Back to the top](#TOP)

## b) available methods <a id="IVb"></a><a name="IVb"></a>

all methods of the `NeptuneConnector` take a `setup_key` argument for specifying "where" the method should happen.

In [ ]:
# stuff you can do with dbs : 

db = nep_con.download_database("db", "demo_db.h5", destination="./")

# "db" is the setup_key, `db` the object to be uploaded

nep_con.upload_database("db", db)

In [ ]:
# "model" is the setup_key, `model` is the object or a path to the root_dir of such an object

# with an object 

nep_con.upload_model("model", model)

# with a path

nep_con.upload_model("model", "experiment-5/version_101")

In [ ]:
# get, create or download experiments with just a setup_key

exp = nep_con.get_experiment("model")

exp = nep_con.create_experiment("model")

# this download all the experiment's artifacts in a folder named after the experiment's id :

exp = nep_con.download_experiment("model")

# download_experiment takes also `artifacts` as argument :

exp = nep_con.download_experiment("model", artifacts="audios/")
exp = nep_con.download_experiment("model", artifacts="states/")

[Back to the top](#TOP)

## c) access `Experiment` object after training <a id="IVc" name="IVc"></a>

In [ ]:
# a model will have 2 loggers when neptune is turned-on and each logger has its own Experiment object.

experiment = model.logger.experiment 
# can then be a list of several Experiment objects....

# if you just want the neptune one :

experiment = model.neptune_experiment

# then, you can log, download, upload what you want :

experiment.log_text("model_class", "FreqNet")

# you will get .zip with the next method. To download & unzip, use `NeptuneConnector.download_experiment`

experiment.download_artifacts("/audios")

[Back to the top](#TOP)

# IV. LOOK AT THE LOGS <a id="IV." name="IV."></a>

## Listen to logged Audios

If you logged audios in tensorboard or neptune, you can listen to them in there respective interfaces.

Otherwise, the audios are still in your path_to_model/audios and you can play them in a notebook like so :

In [ ]:
import os
import librosa
from mimikit.utils import audio

path_to_audios = os.path.join(path_to_model, "audios")

for audio_file in os.listdir(path_to_audios):
    signal, _ = librosa.load(audio_file)
    print(audio_file)
    audio(signal)

[Back to the top](#TOP)